In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [5]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [6]:
#Abiertos

#paginaci´on

link_base_pag = "http://fapesq-prod.codata.pb.gov.br/portal_fapesq/editais/editais-abertos?b_start:int=0"

parser_link_pag = parser(link_base_pag)

ult_page = parser_link_pag.xpath('//ul[@class="paginacao listingBar"]//li/a/@href')[-2].split('int=')[1]

#titulo, link y fecha

link_base = 'http://fapesq-prod.codata.pb.gov.br/portal_fapesq/editais/editais-abertos?b_start:int='


titulos_proyectos = []
links_proyectos = []
fechas_proyectos = []
n = 0

while n <= int(ult_page):
    parser_link = parser(link_base_pag + str(n))

    titulos = parser_link.xpath('//div[@id="content-core"]//article//a/text()')

    links = parser_link.xpath('//div[@id="content-core"]//article//a/@href')

    fechas_brutas = parser_link.xpath('//div[@id="content-core"]//span//span[2]//text()[2]')

    fechas = []

    for f in fechas_brutas:
        fechas.append(f.strip())

    titulos_proyectos+=titulos
    links_proyectos+=links
    fechas_proyectos+=fechas


    n+=10

# PDFs
#     
pdfs_proyectos = []

for link in links_proyectos:
    parser_proyecto = parser(link)

    pdfs_brutos = parser_proyecto.xpath('//div[@id="content-core"]//a/@href')

    pdfs = ''
    for p in pdfs_brutos:
        pdfs = pdfs + p + ', '

    pdfs_proyectos.append(pdfs.strip(', '))

fapesq_abiertos = pd.DataFrame()
fapesq_abiertos['Titulo'] = titulos_proyectos
fapesq_abiertos['Link'] = links_proyectos
fapesq_abiertos['Fecha'] = fechas_proyectos
fapesq_abiertos['Estado'] = 'Abierto'
fapesq_abiertos['Pdfs'] = pdfs_proyectos

In [7]:
#Abiertos

#paginaci´on

link_base_pag = "http://fapesq-prod.codata.pb.gov.br/portal_fapesq/editais/editais-resultados/?b_start:int=0"

parser_link_pag = parser(link_base_pag)

ult_page = parser_link_pag.xpath('//ul[@class="paginacao listingBar"]//li/a/@href')[-2].split('int=')[1]

#titulo, link y fecha

link_base = 'http://fapesq-prod.codata.pb.gov.br/portal_fapesq/editais/editais-resultados/?b_start:int='


titulos_proyectos = []
links_proyectos = []
fechas_proyectos = []
n = 0

while n <= int(ult_page):
    parser_link = parser(link_base_pag + str(n))

    titulos = parser_link.xpath('//div[@id="content-core"]//article//a/text()')

    links = parser_link.xpath('//div[@id="content-core"]//article//a/@href')

    fechas_brutas = parser_link.xpath('//div[@id="content-core"]//span//span[2]//text()[2]')

    fechas = []

    for f in fechas_brutas:
        fechas.append(f.strip())

    titulos_proyectos+=titulos
    links_proyectos+=links
    fechas_proyectos+=fechas


    n+=10

# PDFs
#     
pdfs_proyectos = []

for link in links_proyectos:
    parser_proyecto = parser(link)

    pdfs_brutos = parser_proyecto.xpath('//div[@id="content-core"]//a/@href')

    pdfs = ''
    for p in pdfs_brutos:
        pdfs = pdfs + p + ', '

    pdfs_proyectos.append(pdfs.strip(', '))

fapesq_resultados = pd.DataFrame()
fapesq_resultados['Titulo'] = titulos_proyectos
fapesq_resultados['Link'] = links_proyectos
fapesq_resultados['Fecha'] = fechas_proyectos
fapesq_resultados['Estado'] = 'Resultados'
fapesq_resultados['Pdfs'] = pdfs_proyectos

In [8]:
fapesq = pd.concat([fapesq_abiertos,fapesq_resultados])

fapesq

,Titulo,Link,Fecha,Estado,Pdfs
0,EDITAL Nº 003/2021 – SEECT/FAPESQ/PB - Centená...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,05/05/2021,Abierto,http://fapesq-prod.codata.pb.gov.br/portal_fap...
1,EDITAL Nº 003/2021 – SEECT/FAPESQ/PB - Centená...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,04/05/2021,Abierto,http://fapesq-prod.codata.pb.gov.br/portal_fap...
2,EDITAL Nº 003/2021 – SEECT/FAPESQ/PB - Centená...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,04/05/2021,Abierto,http://fapesq-prod.codata.pb.gov.br/portal_fap...
3,EDITAL Nº 002/2021 PEIEX-PB (APEX-Brasil/FAPES...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,03/05/2021,Abierto,http://fapesq-prod.codata.pb.gov.br/portal_fap...
4,Edital Nº 01/2021 de Apoio a Egressos - Retifi...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,30/04/2021,Abierto,http://fapesq-prod.codata.pb.gov.br/portal_fap...
...,...,...,...,...,...
144,Resultado Final PPSUS 2013,http://fapesq-prod.codata.pb.gov.br/portal_fap...,03/03/2020,Resultados,http://fapesq-prod.codata.pb.gov.br/portal_fap...
145,ICJR 2009 FAPESQ/MCT/CNPq - Alunos Selecionado...,http://fapesq-prod.codata.pb.gov.br/portal_fap...,03/03/2020,Resultados,http://fapesq-prod.codata.pb.gov.br/portal_fap...
146,Resultado Preliminar PAPPE 2011,http://fapesq-prod.codata.pb.gov.br/portal_fap...,03/03/2020,Resultados,http://fapesq-prod.codata.pb.gov.br/portal_fap...
147,Resultado da Seleção PIBIC 2011-2012,http://fapesq-prod.codata.pb.gov.br/portal_fap...,03/03/2020,Resultados,http://fapesq-prod.codata.pb.gov.br/portal_fap...


In [9]:
fapesq.to_excel('brasil_fapesq.xlsx')